In [2]:
from datasets import load_dataset

# load iris
mnist = load_dataset("mnist")

In [6]:
mnistData = mnist['train'].with_format("jax")
X_img = mnistData['image']
y = mnistData['label']


TypeError: unhashable type: 'ArrayImpl'

In [10]:
X = X_img[:, -1]
d_out = len(set(y.tolist()))

In [23]:
from jax import vmap, jit, grad
from jax.nn import sigmoid, softmax
from jax.numpy import zeros
from better_partial import _, partial as F
import sys
sys.path.append('../berries')
import init_utils, random_utils
from init_utils import zerO_init_2D
import optax

def affine(x, W, b):
    return W.T @ x + b

def swish(x):
    return x * sigmoid(x)

def mlp2l(x, W1, b1, W2, b2):
    affineF = F(affine)
    l1 = F(affine)(_, W1, b1)
    l2 = F(affine)(_, W2, b2)
    return l2(swish(l1(x)))

seed = 0
key_gen = random_utils.infinite_safe_keys(seed)

print(X.shape)
n_samples, d_in = X.shape
d_h1 = 128
normal_init_std = 0.1

W1 = init_utils.zerO_init_2D((d_in, d_h1))
#W1 = init_utils.normal_init(next(key_gen), normal_init_std, (d_in, d_h1))
b1 = zeros((d_h1))
W2 = init_utils.zerO_init_2D((d_h1, d_out))
#W2 = init_utils.normal_init(next(key_gen), normal_init_std, (d_h1, d_out))
b2 = zeros((d_out))

mlp2l_b = vmap(mlp2l, in_axes=(0, None, None, None, None), out_axes=0)
mlp2l_b(X, W1, b1, W2, b2)

def sce_loss(to_logits, x, y, W):
    return optax.softmax_cross_entropy_with_integer_labels(to_logits(x, *W), y)


loss_b_all = vmap(F(sce_loss)(mlp2l, _, _, _), (0, 0, None), 0)
loss_d = F(loss_b_all)(X, y, _)
loss_b_d = lambda X: loss_d(X).mean()

W = (W1, b1, W2, b2)
loss0 = loss_b_d(W)
print(loss0)

@jit
def update(W, opt_state):
    grads = grad(loss_b_d)(W)
    updates, opt_state = opt.update(grads, opt_state)
    new_W = optax.apply_updates(W, updates)
    return new_W, opt_state

(60000, 28)
53.69475


In [24]:
lr = 0.01
opt = optax.sgd(lr)
state = opt.init(W)

for i in range(1000):
    W, state = update(W, state)
    if i % 10 == 0:
        print(loss_b_d(W))
print(loss_d(W))

47.56339
11.504119
8.127945
6.672651
6.003635
5.6081753
5.360409
5.1491485
4.9606104
4.7798905
4.604269
4.456606
4.348934
4.244595
4.145376
4.0511403
3.9583771
3.8700686
3.7871058
3.7092257
3.634836
3.5622742
3.4928908
3.4321022
3.3909225
3.3550527
3.326061
3.2978563
3.2699945
3.243211
3.2170558
3.1924303
3.1684875
3.145503
3.1237283
3.1025302
3.0815556
3.0614429
3.0423977
3.0239959
3.005611
2.9872203
2.9696443
2.9528327
2.9361172
2.920027
2.9041724
2.8889418
2.873702
2.8584402
2.8441792
2.8290977
2.814925
2.8008547
2.7870638
2.7731667
2.7596684
2.7465425
2.7328973
2.720114
2.707671
2.696523
2.68606
2.676428
2.666669
2.6577156
2.6508305
2.6444244
2.6382031
2.6321614
2.6256115
2.6189194
2.6127627
2.606761
2.6010509
2.59471
2.58936
2.5832918
2.5776923
2.5726027
2.5670485
2.5617833
2.5565896
2.55141
2.5463471
2.5410688
2.535761
2.5305243
2.5260682
2.5208542
2.5163043
2.5121605
2.5087762
2.5030859
2.4999385
2.4958138
2.4918718
2.4889638
2.484799
2.481763
[2.3927348 2.3034484 2.3171701 ... 